# Baseline Models: Decision Trees

## Objective
Evaluate non-linear tree-based models for credit card fraud detection and
understand the trade-off between model complexity, precision, and recall
under extreme class imbalance.

This notebook compares:
- A **shallow Decision Tree** (controlled complexity)
- A **fully grown Decision Tree** (high variance model)


In [1]:
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import classification_report, confusion_matrix



In [2]:
# Load processed data
X_train = np.load("../data/processed/v1_train_test/X_train_scaled.npy")
X_test  = np.load("../data/processed/v1_train_test/X_test_scaled.npy")
y_train = np.load("../data/processed/v1_train_test/y_train.npy")
y_test  = np.load("../data/processed/v1_train_test/y_test.npy")


In [3]:
def evaluate_classifier(name, y_true, y_pred, y_proba):
    print(f"\n=== {name} ===")
    print("ROC-AUC:", roc_auc_score(y_true, y_proba))
    print("PR-AUC :", average_precision_score(y_true, y_proba))
    print("Confusion Matrix [ [TN FP], [FN TP] ]:")
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred, digits=4, zero_division=0))


## Shallow Decision Tree (max_depth = 5)

A constrained tree is used to:
- limit overfitting
- capture basic non-linear interactions
- serve as a controlled non-linear baseline


In [5]:
tree = DecisionTreeClassifier(
    max_depth=5,
    class_weight="balanced",
    random_state=42
)

tree.fit(X_train, y_train)

tree_pred  = tree.predict(X_test)
tree_proba = tree.predict_proba(X_test)[:, 1]

evaluate_classifier("Decision Tree (max_depth=5)", y_test, tree_pred, tree_proba)



=== Decision Tree (max_depth=5) ===
ROC-AUC: 0.9165621985288207
PR-AUC : 0.44978383369250374
Confusion Matrix [ [TN FP], [FN TP] ]:
[[55135  1729]
 [   12    86]]
              precision    recall  f1-score   support

           0     0.9998    0.9696    0.9845     56864
           1     0.0474    0.8776    0.0899        98

    accuracy                         0.9694     56962
   macro avg     0.5236    0.9236    0.5372     56962
weighted avg     0.9981    0.9694    0.9829     56962



## Fully Grown Decision Tree (No max_depth)

This model allows unrestricted growth and is used to demonstrate:
- high variance behavior
- overfitting risk in imbalanced datasets
- precision–recall trade-offs at extreme complexity


In [8]:
deep_tree = DecisionTreeClassifier(
    max_depth=None,
    class_weight="balanced",
    random_state=42
)

deep_tree.fit(X_train, y_train)

dt_pred  = deep_tree.predict(X_test)
dt_proba = deep_tree.predict_proba(X_test)[:, 1]

evaluate_classifier("Decision Tree (no max_depth)", y_test, dt_pred, dt_proba)



=== Decision Tree (no max_depth) ===
ROC-AUC: 0.8619459390396564
PR-AUC : 0.4903671003078485
Confusion Matrix [ [TN FP], [FN TP] ]:
[[56830    34]
 [   27    71]]
              precision    recall  f1-score   support

           0     0.9995    0.9994    0.9995     56864
           1     0.6762    0.7245    0.6995        98

    accuracy                         0.9989     56962
   macro avg     0.8379    0.8619    0.8495     56962
weighted avg     0.9990    0.9989    0.9989     56962



## Model Comparison & Interpretation

- The shallow tree achieves high recall but suffers from very low precision,
  resulting in many false positives.
- The deep tree significantly improves precision but loses recall,
  indicating overfitting to specific fraud patterns.
- This highlights the instability of single-tree models under severe
  class imbalance.

These results motivate the use of **ensemble methods** such as
Random Forests and Gradient Boosting, which can balance bias and variance
more effectively.
